# **LangSmith 모니터링**: LLM Observability

- LangSmith는 **LLM 애플리케이션의 관찰성(Observability)** 옵션을 제공하는 도구임.

- 개발자가 LLM 체인과 에이전트를 **효과적으로 디버깅**하고 모니터링할 수 있도록 지원함.

- 주요 기능으로는 체인 실행 **로깅 및 추적**이 포함됨.

- **프롬프트 디버깅**과 성능 측정 및 분석 기능을 제공함.

### LangSmith 환경 설정

- langsmith 가입 필요 (https://www.langchain.com/langsmith)
- 유료 (개인 개발자 대상 부분 무료)

- .env 파일에 환경 변수 설정
    ```
    LANGSMITH_TRACING=your_langsmith_api_key
    LANGSMITH_TRACING=true
    LANGSMITH_PROJECT=your_project_name 
    ```

In [1]:
from dotenv import load_dotenv
load_dotenv()

# Langsmith tracing 여부를 확인 
import os
print(os.getenv('LANGSMITH_TRACING'))

true


---

# LCEL(LangChain Expression Language) 

- **LCEL**은 `|` 연산자를 사용해 컴포넌트들을 순차적으로 연결하는 선언적 체이닝을 지원합니다

- **재사용성**이 높아 정의된 체인을 다른 체인의 컴포넌트로 활용할 수 있습니다

- **다양한 실행 방식**(.invoke(), .batch(), .stream(), .astream())으로 동기/비동기 처리가 가능합니다

- **배치 처리**시 자동 최적화를 통해 효율적인 작업 수행이 가능합니다


#### 1) **Prompt + LLM**

* 기본 구조: `Prompt | LLM` 형태로, 파이프(|) 연산자를 사용해 프롬프트와 LLM을 순차적으로 연결합니다.

* 데이터 흐름: 사용자 입력이 Prompt 템플릿을 통해 처리된 후, LLM에 전달되어 최종 응답이 생성됩니다.

* 실행 순서: 파이프라인은 왼쪽에서 오른쪽으로 순차적으로 실행되며, 각 컴포넌트의 출력이 다음 컴포넌트의 입력으로 전달됩니다.

In [7]:
from langchain_openai import ChatOpenAI

# LLM model
llm = ChatOpenAI(
    model="gpt-4.1-nano", 
    temperature=0.3, 
    top_p=0.95,
    )

# 모델에 프롬프트를 입력
response = llm.invoke("탄소의 원자 번호는 무엇인가요?")

print(response.content)

탄소의 원자 번호는 6입니다.


In [8]:
from langchain_core.prompts import PromptTemplate

# 템플릿 문자열 정의
template = """
당신은 {topic} 분야의 전문가입니다. {topic}에 관한 다음 질문에 답변해주세요.
질문: {question}
답변: """

# PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

print("템플릿 변수:")
print(f"- 필수 변수: {prompt.input_variables}")

템플릿 변수:
- 필수 변수: ['question', 'topic']


In [9]:
# chain을 구성
chain = prompt | llm

# chain 실행
response = chain.invoke( 
    {
        "topic": "화학(Chemistry)", 
        "question": "탄소의 원자 번호는 무엇인가요?"
    }
)

print(response.content)

탄소의 원자 번호는 6입니다.


#### **2) Prompt + LLM + Output Parser**

* 데이터 파이프라인: `Prompt | LLM | OutputParser` 형태로 구성되어 LLM의 출력을 구조화된 형식으로 변환합니다.

* Parser 종류: JSON, XML 등 다양한 형식의 파서를 지원하여 LLM 출력을 원하는 데이터 구조로 변환할 수 있습니다.

* 유효성 검증: Parser가 출력 형식을 검증하여 잘못된 형식의 응답을 필터링하고 안정적인 데이터 처리를 보장합니다.

In [ ]:
### 문자열 출력 파서 

from langchain_core.output_parsers import StrOutputParser

# 출력 파서를 생성
output_parser = StrOutputParser() #문자열 파서니까 ai에서 나온 내용을 llm으로 한다.

# 출력 파서를 chain에 추가
chain = prompt | llm | output_parser

# chain을 실행
response = chain.invoke(
    {
        "topic": "화학(Chemistry)", 
        "question": "탄소의 원자 번호는 무엇인가요?"
    }
)

print(response)


탄소의 원자 번호는 6입니다.


---
# **Runnable**

* 실행 인터페이스: 모든 LangChain 컴포넌트는 Runnable 인터페이스를 구현하여 일관된 방식으로 실행됩니다.

* 실행 메서드: `.invoke()`, `.batch()`, `.stream()`, `.astream()` 등 다양한 실행 방식을 제공합니다.

* 호환성: 모든 Runnable 컴포넌트는 파이프(|) 연산자를 통해 연결 가능하며, 재사용이 용이합니다.

* Runnable의 주요 유형:

    * `RunnableSequence`: 여러 Runnable을 순차적으로 실행
    * `RunnablePassthrough`: 입력을 그대로 다음 단계로 전달    
    * `RunnableParallel`: 여러 Runnable을 병렬로 실행
    * `RunnableLambda`: 파이썬 함수를 Runnable로 래핑하여 체인에서 사용, 바로 러너블값으로 사용가능

#### 1) **RunnableSequence**

- **RunnableSequence**는 컴포넌트들을 연결하여 순차적으로 데이터를 처리하는 체인입니다

- `|` 연산자로 연결된 각 단계의 **출력이 다음 단계의 입력**으로 전달됩니다

- **다양한 실행 방식**(동기/비동기, 배치/스트리밍)을 지원합니다

- LLM 체인, 데이터 파이프라인, 자동화된 작업 등 **다단계 처리**에 활용됩니다

In [ ]:
from langchain_core.runnables import RunnableSequence
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


# 컴포넌트 정의
prompt = PromptTemplate.from_template("'{text}'를 영어로 번역해주세요. 번역된 문장만을 출력해주세요.")
model = ChatOpenAI(
    model="gpt-4.1-nano", 
    temperature=0.3, 
    top_p=0.95
)
output_parser = StrOutputParser()

# RunnableSequence 생성 - 함수 사용 
translation_chain = RunnableSequence(
    first=prompt,
    middle=[model],   # 리스트로 전달하는 점에 주의, 미들만 여러개전해주려고 리스트
    last=output_parser
)

# RunnableSequence 생성 - 연산자 사용 파
# translation_chain = prompt | model | output_parser

In [7]:
# 동기 실행
result = translation_chain.invoke({"text": "안녕하세요"})
print(result)  

Hello


#### 2) **RunnableLambda**

- **RunnableLambda**는 일반 함수를 Runnable 객체로 변환하는 래퍼 컴포넌트입니다

- 체인에 **커스텀 로직**을 쉽게 통합할 수 있어 데이터 전처리, 후처리에 유용합니다

- `|` 연산자로 다른 컴포넌트들과 연결해 **복잡한 처리 흐름**을 구성할 수 있습니다

In [11]:
import re #러너블람다가 많잉사용된다. 파이썬함수를 러너블람다의 객체에 랭체인컴포넌트로바꿔버린다.
from langchain_core.runnables import RunnableLambda

# 텍스트에서 숫자를 추출하는 함수 #정수패턴만찾아서 추출해주는.
def extract_number(query):
    return int(re.search(r'\d+', query).group())

# RunnablePassthrough로 입력을 그대로 전달하고, RunnableLambda로 숫자 추출 함수 실행
runnable = RunnableLambda(extract_number)

# 입력 텍스트에서 6을 추출
result = runnable.invoke('탄소의 원자 번호는 6입니다.')
print(result)  # 출력: 6

6


In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage

# 데이터 전처리 함수 정의 입력값 전처리, 후처리 ,평가한다거나
def preprocess_text(text: str) -> str:
    """ 입력 텍스트를 소문자로 변환하고 양쪽 공백을 제거합니다. """
    return text.strip().lower()

# 후처리 함수 정의
def postprocess_response(response: AIMessage) -> dict:
    """ 응답 텍스트를 대문자로 변환하고 길이를 계산합니다. """
    response_text = response.content
    return {
        "processed_response": response_text.upper(),
        "length": len(response_text)
    }
#필요로하는 값들을 구조화해서 출력하는 것들이 중요하다. 가공을해서 출력구조로바꾸는것도 중요하다.

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template("다음 주제에 대해 영어 한 문장으로 설명해주세요: {topic}")

# 처리 파이프라인 구성
chain = (
    RunnableLambda(preprocess_text) |  # 입력 전처리 전체를 러너블람다로 씌운거 러너블 싀퀀스이다. 4개로짜여진.
    prompt |                           # 프롬프트 포맷팅
    llm |                              # LLM 추론
    RunnableLambda(postprocess_response)  # 출력 후처리 
)

# 체인 실행
result = chain.invoke("  Artificial Intelligence  ")
print(f"처리된 응답: {result['processed_response']}")
print(f"응답 길이: {result['length']}")

처리된 응답: ARTIFICIAL INTELLIGENCE IS THE DEVELOPMENT OF COMPUTER SYSTEMS THAT CAN PERFORM TASKS TYPICALLY REQUIRING HUMAN INTELLIGENCE, SUCH AS LEARNING, REASONING, AND PROBLEM-SOLVING.
응답 길이: 175


#### 3) **RunnableParallel**

- **RunnableParallel**은 여러 컴포넌트를 딕셔너리 형태로 구성해 **동시 실행**합니다

- 동일한 입력이 모든 병렬 컴포넌트에 전달되며, 결과는 **키-값 쌍**으로 반환됩니다

- **데이터 변환**과 **파이프라인 구성**에 특화되어 있으며, 출력 형식을 다음 단계에 맞게 조정할 수 있습니다

`(1) 질문 분석 체인`

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Literal

# Pydantic 모델로 구조화된 출력 정의 러너블패러렐은 딕셔너리이다.
class SubjectClassification(BaseModel):
    """질문의 주제 분류 결과""" #셋중에하나 나오게한다 카테고리속성에 출력을해준다.
    category: Literal["화학(Chemistry)", "물리(Physics)", "생물(Biology)"] = Field(
        description="질문이 속하는 과학 분야 카테고리"
    )
    reasoning: str = Field(
        description="분류 이유에 대한 짧은 설명" #모델한테 생각하도록하면 출력품질이 좋아진다. chain of sort와 비슷
    )

# 프롬프트 템플릿 정의 #질문을 세개의 카테고리로 분류한다. 유저메세지로 질문 받고, 모델은 셋중의하나의 형태로 대답한다.
classification_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """당신은 질문을 과학 분야로 분류하는 전문가다.
        질문을 분석하고 해당하는 카테고리로 정확하게 분류해야 한다.
        
        분류 기준:
        - 화학(Chemistry): 원소, 화합물, 반응, 분자 구조 등
        - 물리(Physics): 힘, 운동, 에너지, 파동, 전기 등  
        - 생물(Biology): 생명체, 세포, 유전, 생태계 등"""
    ),
    ("human", "{question}")
])

# 구조화된 출력을 사용한 체인 구성
structured_llm = llm.with_structured_output(SubjectClassification) #이걸로 구조화된 아웃풋나오게한다.
classification_chain = classification_prompt | structured_llm

# 사용 예시
result = classification_chain.invoke({"question": "탄소의 원자 번호는 무엇인가요?"}) #구조화된출력으로 나오게한다.

# 결과는 Pydantic 객체로 반환됨
print(f"분류: {result.category}")
print(f"이유: {result.reasoning}")

분류: 화학(Chemistry)
이유: 탄소의 원자 번호는 원자 번호(프로톤 수)가 6으로, 이는 원소의 특성을 결정하는 핵심 정보이며 화학적 성질과 관련이 깊기 때문에 화학 분야에 속한다.


`(2) 언어 감지 체인`

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Literal, Optional

# Pydantic 모델로 구조화된 출력 정의
class LanguageDetection(BaseModel):
    """텍스트의 언어 감지 결과""" #출력결과를 우리가 설정한 카테고리로 설정가능하다.

    detected_language: Literal["영어(English)", "한국어(Korean)", "스페인어(Spanish)", "중국어(Chinese)", "일본어(Japanese)", "기타(Others)"] = Field(
         description="질문에 사용된 메인 언어"
    )    
    explanation: str = Field(
        description="언어 감지 근거에 대한 짧은 설명"
    )

# 프롬프트 템플릿 정의
language_detection_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """당신은 언어 감지 전문가다. 
        주어진 텍스트의 언어를 정확하게 식별하고 분석해야 한다.
        
        다음 언어들을 주로 구분한다:
        - English: 영어
        - Korean: 한국어
        - Spanish: 스페인어
        - Chinese: 중국어
        - Japanese: 일본어
        - Others: 기타 언어
        
        텍스트의 문자 체계, 단어 패턴, 문법 구조를 분석하여 언어를 판별한다."""
    ),
    ("human", "다음 텍스트의 언어를 감지하세요: {question}")
])

# 구조화된 출력을 사용한 체인 구성
structured_llm = llm.with_structured_output(LanguageDetection)
language_chain = language_detection_prompt | structured_llm

# 사용 예시
examples = [
    "What is the atomic number of carbon?",
    "탄소의 원자 번호는 무엇인가요?",
    "¿Cuál es el número atómico del carbono?",
    "碳的原子序数是多少？",
    "炭素の原子番号は何ですか？",
    "Qual é o número atômico do carbono?"
]

# 각 예시 처리
for example in examples:
    result = language_chain.invoke({"question": example})
    
    print(f"\n입력: {example}")
    print(f"언어: {result.detected_language}")
    print(f"설명: {result.explanation}")
    print(f"--------------------------------")


입력: What is the atomic number of carbon?
언어: 영어(English)
설명: 제공된 텍스트는 영어 단어와 문장 구조를 사용하고 있으며, 영어 특유의 알파벳과 문법 패턴을 보이고 있습니다.
--------------------------------

입력: 탄소의 원자 번호는 무엇인가요?
언어: 한국어(Korean)
설명: 제공된 텍스트는 한글 문자로 구성되어 있으며, '탄소의 원자 번호는 무엇인가요?'라는 문장은 한국어 문법과 어휘를 사용하고 있어 한국어로 판단됩니다.
--------------------------------

입력: ¿Cuál es el número atómico del carbono?
언어: 스페인어(Spanish)
설명: 문장에서 사용된 단어와 문법 구조가 스페인어의 특징을 보이며, '¿'와 '?'로 감싸진 의문문 형식을 갖추고 있어 스페인어임을 확인할 수 있다.
--------------------------------

입력: 碳的原子序数是多少？
언어: 중국어(Chinese)
설명: 제공된 텍스트는 중국어 문자 체계를 사용하며, '原子序数'라는 표현은 화학 원소의 원자번호를 의미하는 중국어 표현입니다. 따라서 이 텍스트는 중국어로 판단됩니다.
--------------------------------

입력: 炭素の原子番号は何ですか？
언어: 중국어(Chinese)
설명: 텍스트에 사용된 문자 체계는 한자(漢字)로, 이는 중국어, 일본어, 또는 한국어에서 사용될 수 있다. 그러나 문장의 구조와 의미를 고려할 때, '炭素の原子番号は何ですか？'는 일본어 문장으로 보이며, 일본어는 일본에서 주로 사용된다. 따라서 이 텍스트는 일본어로 판단된다.
--------------------------------

입력: Qual é o número atômico do carbono?
언어: 스페인어(Spanish)
설명: 제시된 텍스트는 포르투갈어로 보이지만, 포르투갈어와 매우 유사한 스페인어의 문법과 어휘를 포함하

`(3) RunnableParallel을 사용한 병렬 실행 체인`

In [30]:
# 질문과 관련된 분야를 찾아서 질문에 대한 답변을 생성하는 프롬프트
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

# 답변 템플릿 정의
answer_template = """
당신은 {topic} 분야의 전문가입니다. {topic}에 관한 질문에 {language}로 답변해주세요.
질문: {question}
"""

# 프롬프트 및 체인 구성
answer_prompt = PromptTemplate.from_template(answer_template)
output_parser = StrOutputParser()

# 병렬 처리 체인 구성 #세가지를 병렬로 실행한다.
answer_chain = RunnableParallel({
    "topic": classification_chain | RunnableLambda(lambda x: x.category),#이 람다 안의값을 러너블로 매꾸면 바로 컴포넌트로 사용가능함.,            # 주제 분류 체인
    "language": language_chain | RunnableLambda(lambda x: x.detected_language),         # 언어 감지 체인
    "question": itemgetter("question")
    }) | answer_prompt | llm | output_parser 
 #    엔서프롬프트안에 3가지가나오고 llm가고 아웃풋파서지나간다. # 원본 질문 추출 객체로부터의 아래원본질문만 가져오는거다.
# 체인 실행 예시 
result = answer_chain.invoke({
    "question": "탄소의 원자 번호는 무엇인가요?"
})

print("처리 결과:")
print(f"답변: {result}")

처리 결과:
답변: 탄소의 원자 번호는 6입니다.


#### 4) **RunnablePassthrough**

- **RunnablePassthrough**는 입력값을 그대로 전달하여 원본 데이터를 보존합니다

- **RunnableParallel**과 함께 사용되어 입력 데이터를 새로운 키로 매핑할 수 있습니다

- **투명한 데이터 흐름**으로 파이프라인 디버깅과 구성이 용이합니다

In [26]:
from langchain_core.runnables import RunnablePassthrough
import re

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: int(re.search(r'\d+', x["query"]).group())#전달된데이터를 그대로 전달시킨다.,
)

runnable.invoke({"query": '탄소의 원자 번호는 6입니다.'})

{'passed': {'query': '탄소의 원자 번호는 6입니다.'}, 'modified': 6}

In [21]:
runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: int(re.search(r'\d+', x).group()),
)

runnable.invoke('탄소의 원자 번호는 6입니다.')

{'passed': '탄소의 원자 번호는 6입니다.', 'modified': 6}

---
# [실습]

- **다음과 같은 요구사항을 LCEL로 구현합니다**
   1. 사용자 입력을 받아 내용을 요약하기
   2. 요약된 내용을 기반으로 감정 분석하기 (긍정, 부정, 중립)
   3. 요약된 문장과 감정 분석 결과를 출력하기 

In [31]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Literal

# 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ===== 연습문제 1: Pydantic 모델 정의 =====
# TODO: 아래 클래스들의 빈 부분을 채우세요

class Summary(BaseModel):
    """텍스트 요약"""
    one_line_summary: None = Field(  # TODO: str 타입 지정
        description=None  # TODO: "한 문장 요약" 설명 추가
    )
    main_points: list[None] = Field(  # TODO: list[str] 타입 지정
        description="주요 포인트 1~3개"
    )

class SentimentResult(BaseModel):
    """감정 분석 결과"""
    sentiment: None = Field(  # TODO: Literal["긍정", "부정", "중립"] 타입 지정
        description="전반적인 감정"
    )
    intensity: None = Field(  # TODO: float 타입 지정
        description=None  # TODO: "감정의 강도" 설명 추가
    )
    key_phrases: None = Field(  # TODO: list[str] 타입 지정
        description="감정을 나타내는 핵심 표현(최대 5개)"
    )

class TextAnalysis(BaseModel):
    """텍스트 종합 분석"""
    summary: None = Field(  # TODO: Summary 타입 지정
        description="텍스트 요약 정보"
    )
    sentiment: None = Field(  # TODO: SentimentResult 타입 지정
        description=None  # TODO: "감정 분석 결과" 설명 추가
    )

In [32]:
# ===== 연습문제 2: 프롬프트 템플릿 완성 =====
analysis_prompt = ChatPromptTemplate.from_messages([
    (
        None,  # TODO: "system" 역할 지정
        """당신은 텍스트 분석 전문가다.
        주어진 텍스트를 요약하고 감정을 분석한다.
        
        요약 시:
        - 핵심 내용을 한 문장으로 정리
        - 주요 포인트를 정확히 3개 추출
        
        감정 분석 시:
        - 전반적인 감정을 긍정/부정/중립으로 분류
        - 감정의 강도를 0-1 사이 값으로 평가
        - 감정을 나타내는 핵심 표현 추출"""
    ),
    (None, None)  # TODO: "human" 역할과 "다음 텍스트를 분석하세요:\n\n{text}" 메시지
])

TypeError: 'NoneType' object is not callable

In [33]:
# ===== 연습문제 3: 체인 구성 =====
# TODO: with_structured_output 메서드를 사용하여 구조화된 LLM 생성
structured_llm = llm.None(None)  # TODO: with_structured_output(TextAnalysis)

# TODO: 프롬프트와 구조화된 LLM을 파이프(|)로 연결
analysis_chain = None  # TODO: analysis_prompt | structured_llm

SyntaxError: invalid syntax (4059650643.py, line 3)

In [ ]:
# ===== 연습문제 4: 실행 및 결과 처리 =====
text = """오늘 아침 일어나서 시험 준비를 위해 마지막으로 복습을 했다.
시험장에 들어가서 문제를 풀 때는 긴장했지만, 평소에 열심히 공부했던 내용들이 잘 기억났다.
시험 시간이 끝나고 답안지를 제출할 때는 자신감이 있었다.
결과를 확인했을 때 만점을 받았다는 것을 알게 되었고, 그 순간 정말 기뻤다.
지난 몇 주 동안 밤늦게까지 공부했던 노력이 결실을 맺은 것 같아 뿌듯했다.
선생님께서도 칭찬해 주셔서 더욱 기쁘고 보람찼다.
이번 경험을 통해 노력하면 반드시 좋은 결과가 따른다는 것을 다시 한번 깨달았다."""

# TODO: chain의 invoke 메서드를 호출하여 결과 얻기
result = analysis_chain.None({None: None})  # TODO: invoke({"text": text})

In [ ]:
# ===== 연습문제 5: 결과 출력 =====
print("=== 분석 결과 ===")
print(f"\n📝 요약: {result.None.None}")  # TODO: result.summary.one_line_summary

print(f"\n📌 주요 포인트:")
for i, point in enumerate(result.summary.None, 1):  # TODO: main_points 접근
    print(f"  {i}. {point}")

print(f"\n😊 감정: {result.None.None}")  # TODO: result.sentiment.sentiment
print(f"💪 강도: {result.sentiment.None:.2f}")  # TODO: intensity 접근

print(f"\n🔑 핵심 표현:")
for phrase in result.None.None:  # TODO: sentiment.key_phrases 접근
    print(f"  - {phrase}")

<details>
<summary>💡 정답 보기</summary>

```python
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Literal

# 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Pydantic 모델들 정의
class Summary(BaseModel):
    """텍스트 요약"""
    one_line_summary: str = Field(
        description="한 문장 요약"
    )
    main_points: list[str] = Field(
        description="주요 포인트 1~3개",
    )

class SentimentResult(BaseModel):
    """감정 분석 결과"""
    sentiment: Literal["긍정", "부정", "중립"] = Field(
        description="전반적인 감정"
    )
    intensity: float = Field(
        description="감정의 강도"
    )
    key_phrases: list[str] = Field(
        description="감정을 나타내는 핵심 표현(최대 5개)"
    )

class TextAnalysis(BaseModel):
    """텍스트 종합 분석"""
    summary: Summary = Field(
        description="텍스트 요약 정보"
    )
    sentiment: SentimentResult = Field(
        description="감정 분석 결과"
    )

# 프롬프트 템플릿
analysis_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """당신은 텍스트 분석 전문가다.
        주어진 텍스트를 요약하고 감정을 분석한다.
        
        요약 시:
        - 핵심 내용을 한 문장으로 정리
        - 주요 포인트를 정확히 3개 추출
        
        감정 분석 시:
        - 전반적인 감정을 긍정/부정/중립으로 분류
        - 감정의 강도를 0-1 사이 값으로 평가
        - 감정을 나타내는 핵심 표현 추출"""
    ),
    ("human", "다음 텍스트를 분석하세요:\n\n{text}")
])

# 체인 구성
structured_llm = llm.with_structured_output(TextAnalysis)
analysis_chain = analysis_prompt | structured_llm

# 실행
text = """오늘 아침 일어나서 시험 준비를 위해 마지막으로 복습을 했다.
시험장에 들어가서 문제를 풀 때는 긴장했지만, 평소에 열심히 공부했던 내용들이 잘 기억났다.
시험 시간이 끝나고 답안지를 제출할 때는 자신감이 있었다.
결과를 확인했을 때 만점을 받았다는 것을 알게 되었고, 그 순간 정말 기뻤다.
지난 몇 주 동안 밤늦게까지 공부했던 노력이 결실을 맺은 것 같아 뿌듯했다.
선생님께서도 칭찬해 주셔서 더욱 기쁘고 보람찼다.
이번 경험을 통해 노력하면 반드시 좋은 결과가 따른다는 것을 다시 한번 깨달았다."""

result = analysis_chain.invoke({"text": text})

# 결과 출력
print("=== 분석 결과 ===")
print(f"\n📝 요약: {result.summary.one_line_summary}")

print(f"\n📌 주요 포인트:")
for i, point in enumerate(result.summary.main_points, 1):
    print(f"  {i}. {point}")

print(f"\n😊 감정: {result.sentiment.sentiment}")
print(f"💪 강도: {result.sentiment.intensity:.2f}")

print(f"\n🔑 핵심 표현:")
for phrase in result.sentiment.key_phrases:
    print(f"  - {phrase}")
```
</details>
